In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mat_ceng as mc
import numpy as np
import pandas as pd
import math
excel_file = 'column_design_small.xlsx'

### table names
+ Frame Assigns - Sect Prop
+ Frame Assigns - Summary
+ Frame Sec Def - Conc Col Reinf
+ Frame Sec Def - Conc Rect
+ Mat Prop - Basic Mech Props
+ Mat Prop - Concrete Data
+ Mat Prop - Rebar Data
---
+ Design Forces - Columns

In [7]:
df_frames:pd.DataFrame
df_sections:pd.DataFrame

df_frames = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Sect Prop',
                   skiprows=[0, 2], usecols=[2,3,5])
df_frames = df_frames[df_frames['Shape'] == 'Concrete Rectangular']

df = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Summary',
                   skiprows=[0,2],usecols=[2,4])
df_frames = df_frames.merge(df,how='inner', on='UniqueName').drop(axis=1, labels=['Shape'])

df_sections = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Col Reinf',
                   skiprows=[0, 2],usecols=[0,3,6,7,8,10,12])
df_sections = df_sections [df_sections['Reinforcement Configuration'] == 'Rectangular']

df = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Rect',
                   skiprows=[0, 2], usecols=[0,1,3,4,8])
df = df[df['Design Type'] == 'Column']
df_sections = df_sections.merge(df,how='inner', on='Name')

df = pd.read_excel(io=excel_file,sheet_name='Mat Prop - Concrete Data',
                   skiprows=[0, 2], usecols=[0,1])
df_sections = df_sections.merge(df,how='inner', on='Material').drop(axis=1, labels=['Reinforcement Configuration','Material','Design Type'])
df_sections['As'] = (2 * (df_sections['Number Bars 3-Dir'] + df_sections['Number Bars 2-Dir']) - 4) * math.pi / 4 * df_sections['Longitudinal Bar Size'] **2
df_sections['rft_ratio'] = df_sections['As'] / (df_sections['Depth'] * df_sections['Width'])

df_input = df_frames.merge(df_sections,how='inner', left_on='Section Property', right_on='Name').drop(axis=1, labels=['Name'])
df_input

,UniqueName,Section Property,Length,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio
0,4189,C450*1000 (30T25) M,5630,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
1,4188,C450*1000,5630,40,4.0,9.0,25,10,1000.0,450.0,50,10799.224747,0.023998
2,4237,C450*1000,5630,40,4.0,9.0,25,10,1000.0,450.0,50,10799.224747,0.023998
3,5710,C 650*650,13270,40,5.0,5.0,25,10,650.0,650.0,50,7853.981634,0.018589
4,4780,C600*1000,10590,40,5.0,7.0,25,10,1000.0,600.0,50,9817.477042,0.016362
5,4837,C500*1200,13270,40,4.0,8.0,25,10,1200.0,500.0,50,9817.477042,0.016362
6,4681,C500*1200,13270,40,4.0,8.0,25,10,1200.0,500.0,50,9817.477042,0.016362
7,4093,C800*1000,5440,40,7.0,9.0,25,10,1000.0,800.0,50,13744.467859,0.017181
8,4677,C450*800,13270,40,4.0,7.0,20,10,800.0,450.0,50,5654.866776,0.015708
9,4676,C450*750,13270,40,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755


In [8]:
df_forces = pd.read_excel(io=excel_file,sheet_name='Design Forces - Columns',
                   skiprows=[0, 2], usecols=[2,3,4,5,6,7,8,9,10])
df_forces

,UniqueName,Combo,Station,P,V2,V3,T,M2,M3
0,4189,D+L-1,0.00,-389564.72,2491.84,-28269.00,20493805.01,-2.823908e+07,8103856.03
1,4189,D+L-1,838.41,-380132.59,2491.84,-28269.00,20493805.01,-4.362643e+06,6092228.85
2,4189,D+L-1,1676.82,-370700.47,2491.84,-28269.00,20493805.01,1.952727e+07,4076619.84
3,4189,D+L-1,2515.23,-361268.34,2491.84,-28269.00,20493805.01,4.335440e+07,2058347.96
4,4189,D+L-1,3353.64,-351836.22,2491.84,-28269.00,20493805.01,6.704247e+07,38732.16
...,...,...,...,...,...,...,...,...,...
4833,4676,0.9D-1,0.00,-444873.73,6891.55,15399.48,2479082.58,6.121467e+07,42438557.83
4834,4676,0.9D-1,6335.08,-396766.74,6891.55,15399.48,2479082.58,-3.681130e+07,-1171729.42
4835,4676,0.9D-1,8550.00,-379947.17,6891.55,15399.48,2479082.58,-7.045825e+07,-16541101.42
4836,4676,0.9D-1,8550.00,-365263.38,28051.03,98730.05,-4785198.15,1.986502e+08,45811671.38


In [88]:
df_final = df_forces.merge(df_input, how='inner', on='UniqueName')
df_final

,UniqueName,Combo,Station,P,V2,V3,T,M2,M3,Section Property,...,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio
0,4189,D+L-1,0.00,-389564.72,2491.84,-28269.00,20493805.01,-2.823908e+07,8103856.03,C450*1000 (30T25) M,...,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
1,4189,D+L-1,838.41,-380132.59,2491.84,-28269.00,20493805.01,-4.362643e+06,6092228.85,C450*1000 (30T25) M,...,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
2,4189,D+L-1,1676.82,-370700.47,2491.84,-28269.00,20493805.01,1.952727e+07,4076619.84,C450*1000 (30T25) M,...,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
3,4189,D+L-1,2515.23,-361268.34,2491.84,-28269.00,20493805.01,4.335440e+07,2058347.96,C450*1000 (30T25) M,...,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
4,4189,D+L-1,3353.64,-351836.22,2491.84,-28269.00,20493805.01,6.704247e+07,38732.16,C450*1000 (30T25) M,...,40,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,4676,0.9D-1,0.00,-444873.73,6891.55,15399.48,2479082.58,6.121467e+07,42438557.83,C450*750,...,40,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755
4342,4676,0.9D-1,6335.08,-396766.74,6891.55,15399.48,2479082.58,-3.681130e+07,-1171729.42,C450*750,...,40,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755
4343,4676,0.9D-1,8550.00,-379947.17,6891.55,15399.48,2479082.58,-7.045825e+07,-16541101.42,C450*750,...,40,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755
4344,4676,0.9D-1,8550.00,-365263.38,28051.03,98730.05,-4785198.15,1.986502e+08,45811671.38,C450*750,...,40,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755


# calculate betta_nds sustained load ratio

In [94]:
sustained_dict = {
    '1.4D-1':['1.4DL+1.7LL-1','1.4DL+1.7LL-2','UDConS1-1'],
    '1.2D-1':['UDConS2-1', 'UDConS2-2','UDConS3-1', 'UDConS3-2', 'UDConS3-3','UDConS3-4', 'UDConS3-5', 'UDConS3-6', 'UDConS3-7', 'UDConS3-8', 'UDConS3-9', 'UDConS3-10', 'UDConS3-11', 'UDConS3-12',
              'UDConS4-1','UDConS4-2', 'UDConS4-3', 'UDConS4-4', 'UDConS4-5', 'UDConS4-6', 'UDConS4-7', 'UDConS4-8', 'UDConS4-9', 'UDConS4-10', 'UDConS4-11','UDConS4-12',
              'UDConS7-1', 'UDConS7-2', 'UDConS7-3', 'UDConS7-4', 'UDConS7-5',
       'UDConS7-6', 'UDConS8-1', 'UDConS8-2', 'UDConS8-3', 'UDConS8-4',
       'UDConS8-5', 'UDConS8-6'],
    '0.9D-1':['UDConS5-1', 'UDConS5-2', 'UDConS5-3', 'UDConS5-4',
       'UDConS5-5', 'UDConS5-6', 'UDConS5-7', 'UDConS5-8', 'UDConS5-9',
       'UDConS5-10', 'UDConS5-11', 'UDConS5-12', 'UDConS6-1', 'UDConS6-2',
       'UDConS6-3', 'UDConS6-4', 'UDConS6-5', 'UDConS6-6', 'UDConS6-7',
       'UDConS6-8', 'UDConS6-9', 'UDConS6-10', 'UDConS6-11', 'UDConS6-12',
       'UDConS9-1', 'UDConS9-2', 'UDConS9-3',
       'UDConS9-4', 'UDConS9-5', 'UDConS9-6', 'UDConS10-1', 'UDConS10-2',
       'UDConS10-3', 'UDConS10-4', 'UDConS10-5', 'UDConS10-6']
}

def get_betta_dns(df_final:pd.DataFrame, uls_combo:str,sus_combo:str) -> float:
   df_uls = df_final[df_final['Combo'] == uls_combo].set_index('UniqueName').sort_index()['P']
   df_sus = df_final[df_final['Combo'] == sus_combo].set_index('UniqueName').sort_index()['P']
   betta = df_sus / df_uls
   return betta.min()

sustained_dict_value = [{uls_combo:get_betta_dns(df_final,uls_combo,sus_combo) for uls_combo in sustained_dict[sus_combo]} for sus_combo in sustained_dict.keys()]
final_sustained_dict_value = {}
for dic in sustained_dict_value:
   for k,v in dic.items():
      final_sustained_dict_value[k] = min(1.0,v)
final_sustained_dict_value
df_betta_dns = pd.DataFrame(data=[final_sustained_dict_value.keys(),final_sustained_dict_value.values()], index=['Combo','betta_dns']).T
display(df_betta_dns.min(axis=0)['betta_dns'])
df_betta_dns

0.7848694425019836

,Combo,betta_dns
0,1.4DL+1.7LL-1,0.800241
1,1.4DL+1.7LL-2,0.800241
2,UDConS1-1,1.0
3,UDConS2-1,0.784869
4,UDConS2-2,0.784869
...,...,...
72,UDConS10-2,0.967632
73,UDConS10-3,1.0
74,UDConS10-4,0.969282
75,UDConS10-5,0.999237


In [90]:
df = df_final.copy()
df = df.merge(df_betta_dns, how='inner', on='Combo')
df

,UniqueName,Combo,Station,P,V2,V3,T,M2,M3,Section Property,...,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio,betta_dns
0,4189,1.4DL+1.7LL-1,0.00,-555634.69,3389.31,-40457.09,29335495.52,-4.029420e+07,11098044.92,C450*1000 (30T25) M,...,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725,0.800241
1,4189,1.4DL+1.7LL-1,838.41,-542429.71,3389.31,-40457.09,29335495.52,-6.122898e+06,8367686.51,C450*1000 (30T25) M,...,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725,0.800241
2,4189,1.4DL+1.7LL-1,1676.82,-529224.74,3389.31,-40457.09,29335495.52,2.806730e+07,5631859.02,C450*1000 (30T25) M,...,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725,0.800241
3,4189,1.4DL+1.7LL-1,2515.23,-516019.76,3389.31,-40457.09,29335495.52,6.216725e+07,2892352.76,C450*1000 (30T25) M,...,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725,0.800241
4,4189,1.4DL+1.7LL-1,3353.64,-502814.79,3389.31,-40457.09,29335495.52,9.606781e+07,150958.07,C450*1000 (30T25) M,...,5.0,12.0,25,10,1000.0,450.0,50,14726.215564,0.032725,0.800241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076,4676,UDConS10-6,0.00,-439728.54,6427.90,15291.09,2426372.36,6.071850e+07,40098780.35,C450*750,...,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755,0.965987
4077,4676,UDConS10-6,6335.08,-392049.17,6427.90,15291.09,2426372.36,-3.660904e+07,-527612.10,C450*750,...,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755,0.965987
4078,4676,UDConS10-6,8550.00,-375379.11,6427.90,15291.09,2426372.36,-7.001381e+07,-14853653.03,C450*750,...,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755,0.965987
4079,4676,UDConS10-6,8550.00,-361315.70,27661.42,97385.60,-4731944.25,1.956665e+08,45492084.92,C450*750,...,4.0,7.0,20,10,750.0,450.0,50,5654.866776,0.016755,0.965987


In [95]:
df_result = pd.DataFrame()

df_result

""


In [91]:
# sample
'''

mini_df['E'] = mini_df.apply(
    lambda s: concrete_elastic_modulus(s['fc_MPa'], s['gamma']),
    axis=1
)

'''


"\n\nmini_df['E'] = mini_df.apply(\n    lambda s: concrete_elastic_modulus(s['fc_MPa'], s['gamma']),\n    axis=1\n)\n\n"

In [92]:
multiplication = [[i * j for j in range(1, 6)] for i in range(2, 5)]

multiplication

[[2, 4, 6, 8, 10], [3, 6, 9, 12, 15], [4, 8, 12, 16, 20]]